# Advanced Querying Mongo

**⚠️ IMPORTANT: Limit your print to avoid infinite scrolling. Otherwise your
code will be lost between print lines. If working with lists do:**

```python
list(collection.find(query))[:5] #or a reasonably low number
```

Importing libraries and setting up connection

In [14]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
#client.list_database_names()
db = client["ironhack"]
c = db.get_collection("companies")




### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [18]:
# Your Code
filter = {"name": "Babelgum"}
projection ={"name":1}
list(c.find(filter,projection))



[{'_id': ObjectId('52cdef7c4bab8bd675297da0'), 'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [22]:
# Your Code
filter = {"number_of_employees": {"$gte":5000}}
projection ={"name":1}
list(c.find(filter,projection).sort("number_of_employees", -1).limit(20))

[{'_id': ObjectId('52cdef7d4bab8bd67529941a'), 'name': 'Siemens'},
 {'_id': ObjectId('52cdef7c4bab8bd67529856a'), 'name': 'IBM'},
 {'_id': ObjectId('52cdef7d4bab8bd675299d33'), 'name': 'Toyota'},
 {'_id': ObjectId('52cdef7c4bab8bd675297e89'), 'name': 'PayPal'},
 {'_id': ObjectId('52cdef7e4bab8bd67529b0fe'),
  'name': 'Nippon Telegraph and Telephone Corporation'},
 {'_id': ObjectId('52cdef7d4bab8bd675298aa4'), 'name': 'Samsung Electronics'},
 {'_id': ObjectId('52cdef7d4bab8bd675298b99'), 'name': 'Accenture'},
 {'_id': ObjectId('52cdef7e4bab8bd67529a657'),
  'name': 'Tata Consultancy Services'},
 {'_id': ObjectId('52cdef7e4bab8bd67529aa51'),
  'name': 'Flextronics International'},
 {'_id': ObjectId('52cdef7d4bab8bd675299156'), 'name': 'Safeway'},
 {'_id': ObjectId('52cdef7c4bab8bd675297e6f'), 'name': 'Sony'},
 {'_id': ObjectId('52cdef7c4bab8bd675298517'), 'name': 'LG'},
 {'_id': ObjectId('52cdef7d4bab8bd675299d31'), 'name': 'Ford'},
 {'_id': ObjectId('52cdef7d4bab8bd675298b28'), 'name': 

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [25]:
# Your Code
filter = {"$and":[{"founded_year":{"$gte":2000}},{"founded_year":{"$lte":2005}}]}
projection ={"name":1,"founded_year":1}
list(c.find(filter,projection))[:5]

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'founded_year': 2005},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8c'),
  'name': 'Zoho',
  'founded_year': 2005},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8d'),
  'name': 'Digg',
  'founded_year': 2004},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'founded_year': 2004},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8f'),
  'name': 'Omnidrive',
  'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [30]:
# Your Code
filter = {"$and":[{"ipo.valuation_amount":{"$gt":100000000}},{"founded_year":{"$lt":2010}}]}
projection ={"name":1,"ipo":1}
list(c.find(filter,projection))[:5]

[{'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d94'),
  'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297de0'),
  'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297e0c'),
  'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297e7a

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [32]:
# Your Code
filter = {"$and":[{"number_of_employees":{"$lt":1000}},{"founded_year":{"$lt":2005}}]}
projection ={"name":1,"number_of_employees":1}
list(c.find(filter,projection).sort("number_of_employees",-1)[:10])

[{'_id': ObjectId('52cdef7d4bab8bd675298933'),
  'name': 'Infinera Corporation',
  'number_of_employees': 974},
 {'_id': ObjectId('52cdef7e4bab8bd67529ac95'),
  'name': 'NorthPoint Communications Group',
  'number_of_employees': 948},
 {'_id': ObjectId('52cdef7f4bab8bd67529be17'),
  'name': '888 Holdings',
  'number_of_employees': 931},
 {'_id': ObjectId('52cdef7c4bab8bd6752986a2'),
  'name': 'Forrester Research',
  'number_of_employees': 903},
 {'_id': ObjectId('52cdef7e4bab8bd67529af6d'),
  'name': 'SonicWALL',
  'number_of_employees': 900},
 {'_id': ObjectId('52cdef7e4bab8bd67529b21b'),
  'name': 'Webmetrics',
  'number_of_employees': 900},
 {'_id': ObjectId('52cdef7e4bab8bd67529b3bd'),
  'name': 'Cornerstone OnDemand',
  'number_of_employees': 881},
 {'_id': ObjectId('52cdef7c4bab8bd675297ff0'),
  'name': 'Mozilla',
  'number_of_employees': 800},
 {'_id': ObjectId('52cdef7c4bab8bd67529857e'),
  'name': 'Buongiorno',
  'number_of_employees': 800},
 {'_id': ObjectId('52cdef7c4bab8bd6

### 6. All the companies that don't include the `partners` field.

In [33]:
# Your Code
filter = {"partners": {"$exists": False}}
projection ={"name":1}
list(c.find(filter,projection))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [36]:
# Your Code
filter = {"category_code":{"$type": "null"}}
projection ={"name":1,"category_code":1}
list(c.find(filter,projection))[:5]

[{'_id': ObjectId('52cdef7c4bab8bd6752980f6'),
  'name': 'Collective',
  'category_code': None},
 {'_id': ObjectId('52cdef7c4bab8bd675298225'),
  'name': 'Snimmer',
  'category_code': None},
 {'_id': ObjectId('52cdef7c4bab8bd675298226'),
  'name': 'KoolIM',
  'category_code': None},
 {'_id': ObjectId('52cdef7c4bab8bd675298261'),
  'name': 'Level9 Media',
  'category_code': None},
 {'_id': ObjectId('52cdef7c4bab8bd675298262'),
  'name': 'VidKing',
  'category_code': None}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [40]:
# Your Code
filter = {"$and":[{"number_of_employees":{"$lte":1000}},{"number_of_employees":{"$gte":100}}]}
projection ={"name":1,"number_of_employees":1}
list(c.find(filter,projection)[:10])

[{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
  'name': 'AdventNet',
  'number_of_employees': 600},
 {'_id': ObjectId('52cdef7c4bab8bd675297da7'),
  'name': 'AddThis',
  'number_of_employees': 120},
 {'_id': ObjectId('52cdef7c4bab8bd675297da8'),
  'name': 'OpenX',
  'number_of_employees': 305},
 {'_id': ObjectId('52cdef7c4bab8bd675297db5'),
  'name': 'LifeLock',
  'number_of_employees': 644},
 {'_id': ObjectId('52cdef7c4bab8bd675297dbb'),
  'name': 'Jajah',
  'number_of_employees': 110},
 {'_id': ObjectId('52cdef7c4bab8bd675297dc0'),
  'name': 'Livestream',
  'number_of_employees': 120},
 {'_id': ObjectId('52cdef7c4bab8bd675297dc1'),
  'name': 'Ustream',
  'number_of_employees': 250},
 {'_id': ObjectId('52cdef7c4bab8bd675297dda'),
  'name': 'iContact',
  'number_of_employees': 300},
 {'_id': ObjectId('52cdef7c4bab8bd675297de0'),
  'name': 'Yelp',
  'number_of_employees': 800},
 {'_id': ObjectId('52cdef7c4bab8bd675297dee'),
  'name': 'Dailymotion',
  'number_of_employees': 120}]

### 9. Order all the companies by their IPO price in a descending order.

In [42]:
# Your Code
filter = {}
projection ={"name":1,"ipo.valuation_amount":1}
list(c.find(filter,projection).sort("ipo.valuation_amount",-1)[:10])

[{'_id': ObjectId('52cdef7e4bab8bd67529a8b4'),
  'name': 'GREE',
  'ipo': {'valuation_amount': 108960000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297e7a'),
  'name': 'Amazon',
  'ipo': {'valuation_amount': 100000000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d94'),
  'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000}},
 {'_id': ObjectId('52cdef7d4bab8bd675299d5d'),
  'name': 'Groupon',
  'ipo': {'valuation_amount': 12800000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675298674'),
  'name': 'Tencent',
  'ipo': {'valuation_amount': 11000000000}},
 {'_id': ObjectId('52cdef7e4bab8bd67529b996'),
  'name': 'Western Digital',
  'ipo': {'valuation_amount': 9430000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297e0c'),
  'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000}},
 {'_id': ObjectId('52cdef7c4bab8bd67529859a'),
  'name': 'BMC Software',
  'ipo': {'valua

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [43]:
# Your Code
filter = {}
projection ={"name":1,"number_of_employees":1}
list(c.find(filter,projection).sort("number_of_employees",-1)[:11])

[{'_id': ObjectId('52cdef7d4bab8bd67529941a'),
  'name': 'Siemens',
  'number_of_employees': 405000},
 {'_id': ObjectId('52cdef7c4bab8bd67529856a'),
  'name': 'IBM',
  'number_of_employees': 388000},
 {'_id': ObjectId('52cdef7d4bab8bd675299d33'),
  'name': 'Toyota',
  'number_of_employees': 320000},
 {'_id': ObjectId('52cdef7c4bab8bd675297e89'),
  'name': 'PayPal',
  'number_of_employees': 300000},
 {'_id': ObjectId('52cdef7e4bab8bd67529b0fe'),
  'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'_id': ObjectId('52cdef7d4bab8bd675298aa4'),
  'name': 'Samsung Electronics',
  'number_of_employees': 221726},
 {'_id': ObjectId('52cdef7d4bab8bd675298b99'),
  'name': 'Accenture',
  'number_of_employees': 205000},
 {'_id': ObjectId('52cdef7e4bab8bd67529a657'),
  'name': 'Tata Consultancy Services',
  'number_of_employees': 200300},
 {'_id': ObjectId('52cdef7e4bab8bd67529aa51'),
  'name': 'Flextronics International',
  'number_of_employees': 200000},
 {'

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [46]:
# Your Code
filter = {"founded_month":{"$gte":6}}
projection ={"name":1,"founded_month":1}
list(c.find(filter,projection)[:10])#[:1001]) (limiting output to avoid filling the notebook)

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'founded_month': 10},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8c'),
  'name': 'Zoho',
  'founded_month': 9},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8d'),
  'name': 'Digg',
  'founded_month': 10},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8f'),
  'name': 'Omnidrive',
  'founded_month': 11},
 {'_id': ObjectId('52cdef7c4bab8bd675297d90'),
  'name': 'Postini',
  'founded_month': 6},
 {'_id': ObjectId('52cdef7c4bab8bd675297d91'),
  'name': 'Geni',
  'founded_month': 6},
 {'_id': ObjectId('52cdef7c4bab8bd675297d93'),
  'name': 'Fox Interactive Media',
  'founded_month': 6},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9b'),
  'name': 'eBay',
  'founded_month': 9},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9d'),
  'name': 'Joost',
  'founded_month': 10},
 {'_id': ObjectId('52cdef7c4bab8bd675297da1'),
  'name': 'Plaxo',
  'founded_month': 11}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

In [50]:
# Your Code
filter = {"$and":[{"founded_year":{"$lte":2000}},{"acquisition.price_amount":{"$gte":10000000}}]}
projection ={"founded_year":1,"acquisition.price_amount":1}
list(c.find(filter,projection))[:5]

[{'_id': ObjectId('52cdef7c4bab8bd675297d90'),
  'founded_year': 1999,
  'acquisition': {'price_amount': 625000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297deb'),
  'founded_year': 1999,
  'acquisition': {'price_amount': 180000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297e2c'),
  'founded_year': 1999,
  'acquisition': {'price_amount': 25000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297e89'),
  'founded_year': 1998,
  'acquisition': {'price_amount': 1500000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297e97'),
  'founded_year': 1999,
  'acquisition': {'price_amount': 300000000}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [51]:
# Your Code
filter = {"acquisition.acquired_year":{"$gte":2010}}
projection ={"nane":1,"acquisition.acquired_year":1}
list(c.find(filter,projection))[:5]

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'acquisition': {'acquired_year': 2013}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8d'),
  'acquisition': {'acquired_year': 2012}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d91'),
  'acquisition': {'acquired_year': 2012}},
 {'_id': ObjectId('52cdef7c4bab8bd675297dab'),
  'acquisition': {'acquired_year': 2011}},
 {'_id': ObjectId('52cdef7c4bab8bd675297dac'),
  'acquisition': {'acquired_year': 2010}}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [52]:
# Your Code
filter = {}
projection ={"name":1,"founded_year":1}
list(c.find(filter,projection).sort("founded_year",-1)[:11])

[{'_id': ObjectId('52cdef7c4bab8bd67529801f'),
  'name': 'Wamba',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd675297fec'),
  'name': 'Fixya',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7d4bab8bd675299291'),
  'name': 'SEOGroup',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd67529830a'),
  'name': 'Fluc',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7d4bab8bd675298ea7'),
  'name': 'iBazar',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7e4bab8bd67529a3b8'),
  'name': 'Clowdy',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7e4bab8bd67529a6e7'),
  'name': 'WhosCall',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7e4bab8bd67529b40e'),
  'name': 'Tongxue',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7e4bab8bd67529b254'),
  'name': 'Pikk',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7d4bab8bd675298fe1'),
  'name': 'Gimigo',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd6752982d4'),
  'name': 'Advaliant',
  'founded_

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [54]:
# Your Code

filter = {"founded_day":{"$lte":7}}
projection ={"name":1,"founded_day":1}
list(c.find(filter,projection).sort("acquisition.acquisition_price",-1)[:11])

[{'_id': ObjectId('52cdef7c4bab8bd675297d9b'),
  'name': 'eBay',
  'founded_day': 1},
 {'_id': ObjectId('52cdef7c4bab8bd675297da0'),
  'name': 'Babelgum',
  'founded_day': 1},
 {'_id': ObjectId('52cdef7c4bab8bd675297d93'),
  'name': 'Fox Interactive Media',
  'founded_day': 1},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9a'),
  'name': 'Helio',
  'founded_day': 1},
 {'_id': ObjectId('52cdef7c4bab8bd675297d95'),
  'name': 'StumbleUpon',
  'founded_day': 1},
 {'_id': ObjectId('52cdef7c4bab8bd675297d91'),
  'name': 'Geni',
  'founded_day': 1},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9d'),
  'name': 'Joost',
  'founded_day': 1},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8f'),
  'name': 'Omnidrive',
  'founded_day': 1},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'founded_day': 1},
 {'_id': ObjectId('52cdef7c4bab8bd675297d90'),
  'name': 'Postini',
  'founded_day': 2},
 {'_id': ObjectId('52cdef7c4bab8bd675297d96'),
  'name': 'Gizmoz',
  'founded_day': 1}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [56]:
# Your Code
filter = {"$and":[{"number_of_employees":{"$gte":4000}},{"category_code":"web"}]}
projection ={"name":1,"number_of_employees":1}
list(c.find(filter,projection).sort("number_of_employees",1)[:11])

[{'_id': ObjectId('52cdef7c4bab8bd67529822a'),
  'name': 'Expedia',
  'number_of_employees': 4400},
 {'_id': ObjectId('52cdef7c4bab8bd675297e96'),
  'name': 'AOL',
  'number_of_employees': 8000},
 {'_id': ObjectId('52cdef7c4bab8bd675297ea4'),
  'name': 'Webkinz',
  'number_of_employees': 8657},
 {'_id': ObjectId('52cdef7c4bab8bd675297fcb'),
  'name': 'Rakuten',
  'number_of_employees': 10000},
 {'_id': ObjectId('52cdef7c4bab8bd67529834c'),
  'name': 'Los Angeles Times Media Group',
  'number_of_employees': 10000},
 {'_id': ObjectId('52cdef7d4bab8bd675299d5d'),
  'name': 'Groupon',
  'number_of_employees': 10000},
 {'_id': ObjectId('52cdef7c4bab8bd675297da3'),
  'name': 'Yahoo!',
  'number_of_employees': 13600},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9b'),
  'name': 'eBay',
  'number_of_employees': 15000},
 {'_id': ObjectId('52cdef7c4bab8bd6752982a8'),
  'name': 'Experian',
  'number_of_employees': 15500}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [60]:
# Your Code
filter = {"$and":[{"acquisition.price_amount":{"$gte":10000000}},{"acquisition.price_currency_code":"EUR"}]}
projection ={"name":1,"acquisition.price_amount":1}
list(c.find(filter,projection)[:11])

[{'_id': ObjectId('52cdef7c4bab8bd675297f02'),
  'name': 'ZYB',
  'acquisition': {'price_amount': 31500000}},
 {'_id': ObjectId('52cdef7d4bab8bd675298bf3'),
  'name': 'Apertio',
  'acquisition': {'price_amount': 140000000}},
 {'_id': ObjectId('52cdef7d4bab8bd675298f47'),
  'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000}},
 {'_id': ObjectId('52cdef7e4bab8bd67529a536'),
  'name': 'Webedia',
  'acquisition': {'price_amount': 70000000}},
 {'_id': ObjectId('52cdef7e4bab8bd67529a729'),
  'name': 'Wayfinder',
  'acquisition': {'price_amount': 24000000}},
 {'_id': ObjectId('52cdef7e4bab8bd67529b747'),
  'name': 'Tuenti Technologies',
  'acquisition': {'price_amount': 70000000}},
 {'_id': ObjectId('52cdef7f4bab8bd67529c0cf'),
  'name': 'BioMed Central',
  'acquisition': {'price_amount': 43400000}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [61]:
# Your Code
filter = {"acquisition.acquired_month":{"$lte":3}}
projection ={"name":1,"acquisition":1}
list(c.find(filter,projection)[:11])

[{'_id': ObjectId('52cdef7c4bab8bd675297dab'),
  'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'_id': ObjectId('52cdef7c4bab8bd675297db4'),
  'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_descript

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [65]:
# Your Code
filter = {"$and":[{"founded_year":{"$lte":2010}},{"founded_year":{"$lte":2000}},{"acquisition.acquired_year":{"$lte":2011}}]}
projection ={"name":1,"acquisition":1}
list(c.find(filter,projection)[:5])


[{'_id': ObjectId('52cdef7c4bab8bd675297d90'),
  'name': 'Postini',
  'acquisition': {'price_amount': 625000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://www.techcrunch.com/2007/07/09/google-acquires-postini-for-625-million/',
   'source_description': 'http://www.techcrunch.com/2007/07/09/google-acquires-postini-for-625-million/',
   'acquired_year': 2007,
   'acquired_month': 7,
   'acquired_day': 9,
   'acquiring_company': {'name': 'Google', 'permalink': 'google'}}},
 {'_id': ObjectId('52cdef7c4bab8bd675297ddb'),
  'name': 'MeeVee',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.techcrunch.com/2008/05/07/meevee-acquired-by-live-universe/',
   'source_description': '',
   'acquired_year': 2008,
   'acquired_month': 5,
   'acquired_day': None,
   'acquiring_company': {'name': 'LiveUniverse',
    'permalink': 'liveuniverse'}}},
 {'_id': ObjectId('52cdef7c4bab8bd675297deb

### 20. All the companies that have been 'deadpooled' after the third year.

In [66]:
# Your Code
filter = {'$where': 'this.deadpooled_year - this.founded_year > 3'}
projection ={"name":1}
list(c.find(filter,projection)[:5])


[{'_id': ObjectId('52cdef7c4bab8bd675297da0'), 'name': 'Babelgum'},
 {'_id': ObjectId('52cdef7c4bab8bd675297dae'), 'name': 'Thoof'},
 {'_id': ObjectId('52cdef7c4bab8bd675297db3'), 'name': 'Mercora'},
 {'_id': ObjectId('52cdef7c4bab8bd675297db6'), 'name': 'Wesabe'},
 {'_id': ObjectId('52cdef7c4bab8bd675297dbe'), 'name': 'Stickam'}]

**⚠️ Did you do this?**

```python
list(collection.find(query))[:5] #or a reasonably low number
```